In [2]:
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

print(cv2.__version__)

4.1.2


In [14]:
apple = cv2.imread('apple.jpg')
orange = cv2.imread('orange.jpg')

print(apple.shape)
print(orange.shape)
apple_orgh = np.hstack((apple[:, :256], orange[:, 256:]))
apple_orgv = np.vstack((apple[:256, :], orange[256:, :]))


cv2.imshow('apple', apple)
cv2.imshow('orange', orange)
cv2.imshow('apple_orgh', apple_orgh)
cv2.imshow('apple_orgv', apple_orgv)

cv2.waitKey(10000)
cv2.destroyAllWindows()

(512, 512, 3)
(512, 512, 3)


### Image Blending

In [21]:
apple = cv2.imread('apple.jpg')
orange = cv2.imread('orange.jpg')

print(apple.shape)
print(orange.shape)
apple_orgh = np.hstack((apple[:, :256], orange[:, 256:]))
apple_orgv = np.vstack((apple[:256, :], orange[256:, :]))


cv2.imshow('apple', apple)
cv2.imshow('orange', orange)
cv2.imshow('apple_orgh', apple_orgh)
cv2.imshow('apple_orgv', apple_orgv)

cv2.waitKey(10000)
cv2.destroyAllWindows()

(512, 512, 3)
(512, 512, 3)


In [28]:
apple = cv2.imread('apple.jpg')
orange = cv2.imread('orange.jpg')

print(apple.shape)
print(orange.shape)
apple_orange = np.hstack((apple[:, :256], orange[:, 256:]))

# generate gaussian pyramid for apple
apple_copy = apple.copy()
gp_apple = [apple_copy]

for i in range(6):
    apple_copy = cv2.pyrDown(apple_copy)
    gp_apple.append(apple_copy)


# generate gaussian pyramid for Orange
orange_copy = orange.copy()
gp_orange = [orange_copy]

for i in range(6):
    orange_copy = cv2.pyrDown(orange_copy)
    gp_orange.append(orange_copy)
    
# generate laplacian pyramid for apple
apple_copy = gp_apple[5]
lp_apple = [apple_copy]

for i in range(5,0,-1):
    gaussian_expanded = cv2.pyrUp(gp_apple[i])
    laplacian = cv2.subtract(gp_apple[i-1], gaussian_expanded)
    lp_apple.append(laplacian)


# generate laplacian pyramid for Orange
orange_copy = gp_orange[5]
lp_orange = [orange_copy]

for i in range(5,0,-1):
    gaussian_expanded = cv2.pyrUp(gp_orange[i])
    laplacian = cv2.subtract(gp_orange[i-1], gaussian_expanded)
    lp_orange.append(laplacian)
    
# left and right halves of image in each level
apple_orange_pyramid =[]
n= 0

for apple_lap, orange_lap in zip(lp_apple,lp_orange):
    n +=1
    cols, rows, ch = apple_lap.shape
    laplacian = np.hstack((apple_lap[:,  :int(cols/2)],orange_lap[:,  int(cols/2):] ))
    apple_orange_pyramid.append(laplacian)

# reconstruct
apple_orange_reconstruct = apple_orange_pyramid[0]

for i in range(1, 6):
    apple_orange_reconstruct = cv2.pyrUp(apple_orange_reconstruct)
    apple_orange_reconstruct = cv2.add(apple_orange_pyramid[i], apple_orange_reconstruct)


cv2.imshow('apple', apple)
cv2.imshow('orange', orange)
cv2.imshow('apple_orange', apple_orange)
cv2.imshow('apple_orange_reconstruct', apple_orange_reconstruct)

cv2.waitKey(10000)
cv2.destroyAllWindows()

(512, 512, 3)
(512, 512, 3)


## Basic Motion Detection

In [1]:
import cv2
import numpy as np


cap = cv2.VideoCapture(0)#'C:/Users/Windows 10/Desktop/DL/CV/opencv-master/samples/data/vtest.avi')
ret, frame1 =cap.read()
ret, frame2 =cap.read()


while cap.isOpened():
    diff =cv2.absdiff(frame1, frame2)
    gray =cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    blur =cv2.GaussianBlur(gray, (5,5), 0)
    _, thresh =cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
    dilated =cv2.dilate(thresh, None, iterations=3)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    
    cv2.drawContours(frame1, contours, -1, (0, 255, 0), 2)
    
    cv2.imshow("inter", frame1)
    frame1= frame2
    ret, frame2 =cap.read()
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows

<function destroyAllWindows>

In [1]:
import cv2
import numpy as np


cap = cv2.VideoCapture(0) #'C:/Users/Windows 10/Desktop/DL/CV/opencv-master/samples/data/vtest.avi')
ret, frame1 =cap.read()
ret, frame2 =cap.read()


while cap.isOpened():
    diff =cv2.absdiff(frame1, frame2)
    gray =cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    blur =cv2.GaussianBlur(gray, (5,5), 0)
    _, thresh =cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
    dilated =cv2.dilate(thresh, None, iterations=3)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    
    
    for contour in contours:
        (x, y, w, h) =cv2.boundingRect(contour)
        
        if cv2.contourArea(contour) < 700:
            continue 
        cv2.rectangle(frame1, (x,y), (x+w, y+w), (0,255,0) , 2)
        cv2.putText(frame1, "Status: {}".format('Movement') ,(10, 20),  cv2.FONT_HERSHEY_SIMPLEX, 1 , (0,0,255) , 3)
    
    
    
    cv2.imshow("inter", frame1)
    frame1= frame2
    ret, frame2 =cap.read()
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows

<function destroyAllWindows>

### Shape detection

In [7]:
img = cv2.imread('pic3.png')
imgGray =cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

cv2.imshow('shapes', img)
cv2.waitKey(10000)
cv2.destroyAllWindows()

In [15]:
# Load the image
img = cv2.imread("cont_new.png")

        
cv2.imshow("Image", img)       

# Convert to greyscale
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# Convert to binary image by thresholding
_, threshold = cv2.threshold(img_gray, 245, 255, cv2.THRESH_BINARY)
# Find the contours
contours, _ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# For each contour approximate the curve and
# detect the shapes.
for cnt in contours:
    epsilon = 0.01*cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, epsilon, True)
    cv2.drawContours(img, [approx], 0, (0), 2)
    # Position for writing text
    x = approx.ravel()[0]
    y = approx.ravel()[1]
 
    if len(approx) == 3:
        cv2.putText(img, "Triangle", (x, y), cv2.FONT_HERSHEY_COMPLEX, 1, 0,2)
    elif len(approx) == 4:
        cv2.putText(img, "Rectangle", (x, y), cv2.FONT_HERSHEY_COMPLEX, 1, 0,2)
    elif len(approx) == 5:
        cv2.putText(img, "Pentagon", (x, y), cv2.FONT_HERSHEY_COMPLEX, 1, 0,2)
    elif 6 < len(approx) < 15:
        cv2.putText(img, "Ellipse", (x, y), cv2.FONT_HERSHEY_COMPLEX, 1, 0,2)
    else:
        cv2.putText(img, "Circle", (x, y), cv2.FONT_HERSHEY_COMPLEX, 1, 0,2)
cv2.imshow("Final", img)
cv2.waitKey(10000)
cv2.destroyAllWindows()

### Circle Detection

In [ ]:
img = cv2.imread('smarties.png')
output=img.copy()


gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.medianBlur(gray, 5)

circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, 20,
                               param1=100, param2=30,
                               minRadius=1, maxRadius=30)

detected_circles = np.uint16(np.around(circles))

for (x, y, r) in detected_circles[0, :]:
    cv2.circle(output, (x,y) , r, (0,255,0), 3)
    cv2.circle(output, (x,y) , 2, (0,255,255), 3)



cv2.imshow('output', output)
cv2.waitKey(10000)
cv2.destroyAllWindows()